In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
from collections import Counter
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
import time

In [2]:
df = pd.read_csv('../../Data/paths_clean_AOA_Data.csv')

C:\Users\Anh Tran\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (501,502,503,513,514,515,522,530,538,546,554,562) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,id,client_id,case_number,assign_id,new,new_3,as_recnum,as_date_opened,as_date_closed,as_date_prev,...,regional,vet,mar,as_gap_since_former_IP,IP_history,fiscal_year,svc_AOA_ip_7days_postdc_bi,svc_AOA_ip_30days_postdc_bi,svc_AOA_ip_60days_postdc_bi,svc_AOA_ip_90days_postdc_bi
0,1,2,15112001,1348733,0,0,1379323,2018-09-04,2018-10-04,NaN,...,NaN,0.0,Never Married,NaN,0.0,FY 2018-2019,1,1,1,1
1,2,2,15112001,1362425,0,0,1393015,2018-10-10,2018-10-29,2018-09-04,...,NaN,0.0,Never Married,36.0,1.0,FY 2018-2019,0,0,0,0
2,3,20,13997401,285747,0,0,286469,2010-08-28,2010-09-09,NaN,...,NaN,NaN,NaN,NaN,0.0,FY 2010-2011,0,0,0,0
3,4,20,13997401,569153,0,0,570150,2012-12-16,2012-12-20,2010-08-28,...,NaN,NaN,NaN,841.0,1.0,FY 2012-2013,0,0,0,0
4,5,23,100196481,536063,0,0,536966,2012-09-12,2012-09-14,NaN,...,NaN,NaN,NaN,NaN,0.0,FY 2012-2013,0,0,0,0


## Define variables used for the models

In [4]:
# Initial variables for the model. 
# I focus exclusively on the pre services and loc plus demographics with less than 20% of missing data.

id_cols = ['id', 'client_id']

demo_cols = ['age', 'age_group','race','gender','dx_group','lang', 'substance','insurance','emp', 'educ','liv','justice', 'sexorient', 'co']

ip_cols = ['as_loc', 'as_length_IP', 'IP_history', 'as_close']

loc_pre_cols = ['loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre']

svc_total_pre_cols = ['new', 'svc_pre_all', 'svc_pre', 'svc_pre_sub']

svc_AOA_timepoint_pre_cols = ['svc_AOA_op_7days_pre', 'svc_AOA_op_30days_pre', 'svc_AOA_op_60days_pre', 'svc_AOA_op_90days_pre',
                     'svc_AOA_es_7days_pre', 'svc_AOA_es_30days_pre', 'svc_AOA_es_60days_pre', 'svc_AOA_es_90days_pre',
                     'svc_AOA_24h_7days_pre', 'svc_AOA_24h_30days_pre', 'svc_AOA_24h_60days_pre', 'svc_AOA_24h_90days_pre',
                     'svc_AOA_ip_7days_pre', 'svc_AOA_ip_30days_pre', 'svc_AOA_ip_60days_pre', 'svc_AOA_ip_90days_pre']



output_cols = ['svc_AOA_ip_7days_postdc_bi', 'svc_AOA_ip_30days_postdc_bi', 'svc_AOA_ip_60days_postdc_bi', 'svc_AOA_ip_90days_postdc_bi']
input_cols = id_cols + demo_cols +  ip_cols + loc_pre_cols + svc_total_pre_cols  + svc_AOA_timepoint_pre_cols



In [5]:
df = df[input_cols + output_cols]
df.head()

,id,client_id,age,age_group,race,gender,dx_group,lang,substance,insurance,...,svc_AOA_24h_60days_pre,svc_AOA_24h_90days_pre,svc_AOA_ip_7days_pre,svc_AOA_ip_30days_pre,svc_AOA_ip_60days_pre,svc_AOA_ip_90days_pre,svc_AOA_ip_7days_postdc_bi,svc_AOA_ip_30days_postdc_bi,svc_AOA_ip_60days_postdc_bi,svc_AOA_ip_90days_postdc_bi
0,1,2,50,Age 26-59,White,Male,Schizophrenia and Other Psychotic Disorders,English,1.0,Medi-Cal + Medicare,...,0,0,0,0,0,0,1,1,1,1
1,2,2,50,Age 26-59,White,Male,Schizophrenia and Other Psychotic Disorders,English,1.0,Medi-Cal + Medicare,...,0,0,0,0,1,1,0,0,0,0
2,3,20,57,Age 26-59,White,Male,Bipolar Disorders,English,1.0,Uninsured/Unknown,...,0,0,0,0,0,0,0,0,0,0
3,4,20,60,Age 60+,White,Male,Bipolar Disorders,English,1.0,Uninsured/Unknown,...,0,0,0,0,0,0,0,0,0,0
4,5,23,28,Age 26-59,White,Male,NaN,English,0.0,Medi-Cal + Medicare,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Recode some missing data
df.loc1_pre.fillna("None", inplace=True)
df.loc2_pre.fillna("None", inplace=True)
df.loc3_pre.fillna("None", inplace=True)
df.loc4_pre.fillna("None", inplace=True)
df.loc5_pre.fillna("None", inplace=True)


In [7]:
# Set a seed for reproducibility
SEED = 100
np.random.seed(SEED)
    
# create numpy arrays with the explanatory variables and output label
y = df[['id', 'svc_AOA_ip_7days_postdc_bi', 'svc_AOA_ip_30days_postdc_bi', 'svc_AOA_ip_60days_postdc_bi', 'svc_AOA_ip_90days_postdc_bi']]
X = df[input_cols]
    
# split the data into 80% train and 20% test, scale all variables between 0 and 1
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=SEED, stratify=y['svc_AOA_ip_90days_postdc_bi'])


In [8]:
Xtrain = Xtrain.sort_index()
Xtest = Xtest.sort_index()
ytrain = ytrain.sort_index()
ytest = ytest.sort_index()

In [9]:
ytrain.isin([np.nan, np.inf, -np.inf]).any()

id                             False
svc_AOA_ip_7days_postdc_bi     False
svc_AOA_ip_30days_postdc_bi    False
svc_AOA_ip_60days_postdc_bi    False
svc_AOA_ip_90days_postdc_bi    False
dtype: bool

In [10]:
# Recode categorical variables to numerical code - training dataset
original = Xtrain.copy()
mask = original.isnull()
Xtrain = Xtrain.astype(str).apply(LabelEncoder().fit_transform)
Xtrain = Xtrain.where(~mask, original)
Xtrain = Xtrain.astype('Int64')

mask = mask.sort_index()
original = original.sort_index()

In [11]:
# Recode categorical variables to numerical code - testing dataset
original2 = Xtest.copy()
mask2 = original2.isnull()
Xtest = Xtest.astype(str).apply(LabelEncoder().fit_transform)
Xtest = Xtest.where(~mask2, original2)
Xtest = Xtest.astype('Int64')

mask2 = mask2.sort_index()
original2 = original2.sort_index()

In [12]:
# Train dataset
print(original[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print(Xtrain[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print('##############')

print(original[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print(Xtrain[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print('##############')

print(original[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print(Xtrain[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print('##############')

print(original[['client_id','emp']].groupby('client_id').max().emp.value_counts())
print(Xtrain[['client_id','emp']].groupby('client_id').max().emp.value_counts())
print('##############')

print(original[['client_id','educ']].groupby('client_id').max().educ.value_counts())
print(Xtrain[['client_id','educ']].groupby('client_id').max().educ.value_counts())
print('##############')

print(original[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print(Xtrain[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print('##############')

print(original[['client_id','sexorient']].groupby('client_id').max().sexorient.value_counts())
print(Xtrain[['client_id','sexorient']].groupby('client_id').max().sexorient.value_counts())
print('##############')

Schizophrenia and Other Psychotic Disorders    13959
Depressive Disorders                            7898
Bipolar Disorders                               6738
Other/ Excluded                                 1008
Stressor and Adjustment Disorders                486
Anxiety Disorders                                307
Name: dx_group, dtype: int64
4    13959
2     7898
1     6738
3     1008
5      486
0      307
Name: dx_group, dtype: Int64
##############
English                     28055
Other/Unknown                2809
Spanish                      1844
Asian languages               359
Middle Eastern languages      198
Name: lang, dtype: int64
1    28116
3     2820
4     1851
0      359
2      198
Name: lang, dtype: Int64
##############
Medi-Cal + Medicare    22380
Uninsured/Unknown       9347
Private                 1538
Name: insurance, dtype: int64
0    22426
2     9376
1     1542
Name: insurance, dtype: Int64
##############
Not in Labor Force                10190
Unknown           

In [13]:
# Test dataset
print(original2[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print(Xtest[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print('##############')

print(original2[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print(Xtest[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print('##############')

print(original2[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print(Xtest[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print('##############')

print(original2[['client_id','emp']].groupby('client_id').max().emp.value_counts())
print(Xtest[['client_id','emp']].groupby('client_id').max().emp.value_counts())
print('##############')

print(original2[['client_id','educ']].groupby('client_id').max().educ.value_counts())
print(Xtest[['client_id','educ']].groupby('client_id').max().educ.value_counts())
print('##############')

print(original2[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print(Xtest[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print('##############')

print(original2[['client_id','sexorient']].groupby('client_id').max().sexorient.value_counts())
print(Xtest[['client_id','sexorient']].groupby('client_id').max().sexorient.value_counts())
print('##############')

Schizophrenia and Other Psychotic Disorders    6267
Bipolar Disorders                              2724
Depressive Disorders                           2717
Other/ Excluded                                 391
Stressor and Adjustment Disorders               142
Anxiety Disorders                               117
Name: dx_group, dtype: int64
4    6267
1    2724
2    2717
3     391
5     142
0     117
Name: dx_group, dtype: Int64
##############
English                     11469
Other/Unknown                 831
Spanish                       679
Asian languages               149
Middle Eastern languages       71
Name: lang, dtype: int64
1    11474
3      832
4      680
0      149
2       71
Name: lang, dtype: Int64
##############
Medi-Cal + Medicare    9423
Uninsured/Unknown      3132
Private                 644
Name: insurance, dtype: int64
0    9426
2    3134
1     646
Name: insurance, dtype: Int64
##############
Not in Labor Force                4411
Not Seeking Work                  282

### Data imputation for training dataset

In [14]:
# Check data type
var_name = list(Xtrain.columns.values)
type_name = list(Xtrain.dtypes)
var_list = pd.DataFrame(
    {'variable_name': var_name,
     'data_type': type_name
    })
pd.set_option('display.max_rows', None)
print(var_list)

             variable_name data_type
0                       id     Int64
1                client_id     Int64
2                      age     Int64
3                age_group     Int64
4                     race     Int64
5                   gender     Int64
6                 dx_group     Int64
7                     lang     Int64
8                substance     Int64
9                insurance     Int64
10                     emp     Int64
11                    educ     Int64
12                     liv     Int64
13                 justice     Int64
14               sexorient     Int64
15                      co     Int64
16                  as_loc     Int64
17            as_length_IP     Int64
18              IP_history     Int64
19                as_close     Int64
20                loc1_pre     Int64
21                loc2_pre     Int64
22                loc3_pre     Int64
23                loc4_pre     Int64
24                loc5_pre     Int64
25                     new     Int64
2

In [15]:
# count number of rows with missing values
n_miss = Xtrain.isnull().sum()
perc = Xtrain.isnull().sum()/len(Xtrain)*100
df_missing = pd.merge(n_miss.rename('n_miss'), perc.rename('perc_miss'), right_index = True,
               left_index = True)
pd.set_option('display.max_rows', None)
print(df_missing)

                        n_miss  perc_miss
id                           0   0.000000
client_id                    0   0.000000
age                          0   0.000000
age_group                    0   0.000000
race                         0   0.000000
gender                       0   0.000000
dx_group                  4071   4.850124
lang                       384   0.457491
substance                  392   0.467022
insurance                  384   0.457491
emp                        392   0.467022
educ                       392   0.467022
liv                        392   0.467022
justice                      0   0.000000
sexorient                17344  20.663363
co                         392   0.467022
as_loc                       0   0.000000
as_length_IP                 0   0.000000
IP_history                   0   0.000000
as_close                     0   0.000000
loc1_pre                     0   0.000000
loc2_pre                     0   0.000000
loc3_pre                     0   0

In [16]:
mis_dat_cols = ['dx_group','lang','insurance','emp','educ','liv','sexorient', 'substance','co']

In [17]:
pd.set_option('display.max_columns', None)
Xtrain.head()

,id,client_id,age,age_group,race,gender,dx_group,lang,substance,insurance,emp,educ,liv,justice,sexorient,co,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_AOA_op_7days_pre,svc_AOA_op_30days_pre,svc_AOA_op_60days_pre,svc_AOA_op_90days_pre,svc_AOA_es_7days_pre,svc_AOA_es_30days_pre,svc_AOA_es_60days_pre,svc_AOA_es_90days_pre,svc_AOA_24h_7days_pre,svc_AOA_24h_30days_pre,svc_AOA_24h_60days_pre,svc_AOA_24h_90days_pre,svc_AOA_ip_7days_pre,svc_AOA_ip_30days_pre,svc_AOA_ip_60days_pre,svc_AOA_ip_90days_pre
0,0,3127,52,0,6,1,4,1,1,0,1,3,4,1,2,1,1,167,0,2,34,13,24,24,22,0,521,123,0,0,0,0,0,13,23,23,23,0,0,0,0,0,0,0,0
1,20702,3127,52,0,6,1,4,1,1,0,1,3,4,1,2,1,1,103,1,0,17,34,14,24,22,0,523,1,1,0,0,0,0,0,0,23,23,0,0,0,0,0,0,1,1
2,28753,3128,59,0,6,1,1,1,1,2,2,3,4,0,<NA>,1,0,28,0,7,7,26,24,24,22,0,862,398,0,0,0,0,33,14,28,34,34,0,0,0,0,0,0,0,0
3,37467,3128,63,1,6,1,1,1,1,2,2,3,4,0,2,1,0,202,1,7,7,26,24,24,22,0,241,582,1,0,0,1,1,13,23,23,23,0,0,0,0,0,0,0,0
4,45848,4349,30,0,6,1,<NA>,1,0,0,2,3,4,0,2,0,1,113,0,21,26,13,24,24,22,0,325,62,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
Xtrain = Xtrain.drop(['id', 'client_id'], axis = 1)

In [19]:
Xtrain.shape[0]

83936

In [20]:
k = math.sqrt(Xtrain.shape[0])/2
k

144.85855169785455

In [21]:
#Imputing data with KNN
imputer = KNNImputer(n_neighbors=145, weights='uniform', metric='nan_euclidean')
Xtrain = pd.DataFrame(imputer.fit_transform(Xtrain),columns = Xtrain.columns)

In [22]:
Xtrain = Xtrain.astype(int)
Xtrain.head()

,age,age_group,race,gender,dx_group,lang,substance,insurance,emp,educ,liv,justice,sexorient,co,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_AOA_op_7days_pre,svc_AOA_op_30days_pre,svc_AOA_op_60days_pre,svc_AOA_op_90days_pre,svc_AOA_es_7days_pre,svc_AOA_es_30days_pre,svc_AOA_es_60days_pre,svc_AOA_es_90days_pre,svc_AOA_24h_7days_pre,svc_AOA_24h_30days_pre,svc_AOA_24h_60days_pre,svc_AOA_24h_90days_pre,svc_AOA_ip_7days_pre,svc_AOA_ip_30days_pre,svc_AOA_ip_60days_pre,svc_AOA_ip_90days_pre
0,52,0,6,1,4,1,1,0,1,3,4,1,2,1,1,167,0,2,34,13,24,24,22,0,521,123,0,0,0,0,0,13,23,23,23,0,0,0,0,0,0,0,0
1,52,0,6,1,4,1,1,0,1,3,4,1,2,1,1,103,1,0,17,34,14,24,22,0,523,1,1,0,0,0,0,0,0,23,23,0,0,0,0,0,0,1,1
2,59,0,6,1,1,1,1,2,2,3,4,0,1,1,0,28,0,7,7,26,24,24,22,0,862,398,0,0,0,0,33,14,28,34,34,0,0,0,0,0,0,0,0
3,63,1,6,1,1,1,1,2,2,3,4,0,2,1,0,202,1,7,7,26,24,24,22,0,241,582,1,0,0,1,1,13,23,23,23,0,0,0,0,0,0,0,0
4,30,0,6,1,2,1,0,0,2,3,4,0,2,0,1,113,0,21,26,13,24,24,22,0,325,62,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
Xtrain.isnull().values.any()

False

In [24]:
Xtrain_imputedcols = Xtrain[mis_dat_cols]
Xtrain_othercols = original.drop([x for x in mis_dat_cols], axis = 1)
Xtrain_imputedcols = Xtrain_imputedcols.reset_index(drop=True)
Xtrain_othercols = Xtrain_othercols.reset_index(drop=True)

In [25]:
Xtrain_copy = Xtrain.copy()
df2 = Xtrain_othercols.join(Xtrain_imputedcols)
print(df2.shape)
print(df2.isnull().values.any())

(83936, 45)
False


In [26]:
df2['dx_group'] = df2['dx_group'].replace(0,'Anxiety Disorders')
df2['dx_group'] = df2['dx_group'].replace(1,'Bipolar Disorders')
df2['dx_group'] = df2['dx_group'].replace(2,'Depressive Disorders')
df2['dx_group'] = df2['dx_group'].replace(3,'Other/ Excluded')
df2['dx_group'] = df2['dx_group'].replace(4,'Schizophrenia and Other Psychotic Disorders')
df2['dx_group'] = df2['dx_group'].replace(5,'Stressor and Adjustment Disorders')

df2['lang'] = df2['lang'].replace(0,'Asian languages')
df2['lang'] = df2['lang'].replace(1,'English')
df2['lang'] = df2['lang'].replace(2,'Middle Eastern languages')
df2['lang'] = df2['lang'].replace(3,'Other/Unknown')
df2['lang'] = df2['lang'].replace(4,'Spanish')

df2['insurance'] = df2['insurance'].replace(0,'Medi-Cal + Medicare')
df2['insurance'] = df2['insurance'].replace(1,'Private')
df2['insurance'] = df2['insurance'].replace(2,'Uninsured/Unknown')

df2['emp'] = df2['emp'].replace(0,'Competitive Job')
df2['emp'] = df2['emp'].replace(1,'Not Seeking Work')
df2['emp'] = df2['emp'].replace(2,'Not in Labor Force')
df2['emp'] = df2['emp'].replace(3,'Other')
df2['emp'] = df2['emp'].replace(4,'Resident/Inmate of Institution')
df2['emp'] = df2['emp'].replace(5,'Seeking Work')
df2['emp'] = df2['emp'].replace(6,'Unknown')

df2['educ'] = df2['educ'].replace(0,'Associates Degree')
df2['educ'] = df2['educ'].replace(1,'Bachelors Degree')
df2['educ'] = df2['educ'].replace(2,'Doctoral Degree')
df2['educ'] = df2['educ'].replace(3,'High School Diploma/GED')
df2['educ'] = df2['educ'].replace(4,'High School Not Completed')
df2['educ'] = df2['educ'].replace(5,'Masters Degree')
df2['educ'] = df2['educ'].replace(6,'Some College/Vocational Training')
df2['educ'] = df2['educ'].replace(7,'Unknown/Not Reported')

df2['liv'] = df2['liv'].replace(0,'Board & Care')
df2['liv'] = df2['liv'].replace(1,'Homeless')
df2['liv'] = df2['liv'].replace(2,'Institutional')
df2['liv'] = df2['liv'].replace(3,'Justice Related')
df2['liv'] = df2['liv'].replace(4,'Lives Independently')
df2['liv'] = df2['liv'].replace(5,'Other/Unknown')
                                  
df2['sexorient'] = df2['sexorient'].replace(0,'Heterosexual')
df2['sexorient'] = df2['sexorient'].replace(1,'LGBTQ+')
df2['sexorient'] = df2['sexorient'].replace(2,'Missing/Not Reported')

In [27]:
df2.head()

,id,client_id,age,age_group,race,gender,justice,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_AOA_op_7days_pre,svc_AOA_op_30days_pre,svc_AOA_op_60days_pre,svc_AOA_op_90days_pre,svc_AOA_es_7days_pre,svc_AOA_es_30days_pre,svc_AOA_es_60days_pre,svc_AOA_es_90days_pre,svc_AOA_24h_7days_pre,svc_AOA_24h_30days_pre,svc_AOA_24h_60days_pre,svc_AOA_24h_90days_pre,svc_AOA_ip_7days_pre,svc_AOA_ip_30days_pre,svc_AOA_ip_60days_pre,svc_AOA_ip_90days_pre,dx_group,lang,insurance,emp,educ,liv,sexorient,substance,co
0,1,2,50,Age 26-59,White,Male,1,IPFFS,30,0.0,Board & Care,UO,FFS,None,None,None,0,191,191,0,0,0,0,0,3,3,3,3,0,0,0,0,0,0,0,0,Schizophrenia and Other Psychotic Disorders,English,Medi-Cal + Medicare,Not Seeking Work,High School Diploma/GED,Lives Independently,Missing/Not Reported,1,1
1,2,2,50,Age 26-59,White,Male,1,IPFFS,19,1.0,"Unplanned, AMA or\r\nUnknown",IPFFS,UO,FFS,None,None,0,192,1,1,0,0,0,0,0,0,3,3,0,0,0,0,0,0,1,1,Schizophrenia and Other Psychotic Disorders,English,Medi-Cal + Medicare,Not Seeking Work,High School Diploma/GED,Lives Independently,Missing/Not Reported,1,1
2,3,20,57,Age 26-59,White,Male,0,IPCNTY,12,0.0,Home/Shelter/Self,CS,OP,None,None,None,0,46,46,0,0,0,0,2,4,4,4,4,0,0,0,0,0,0,0,0,Bipolar Disorders,English,Uninsured/Unknown,Not in Labor Force,High School Diploma/GED,Lives Independently,LGBTQ+,1,1
3,4,20,60,Age 60+,White,Male,0,IPCNTY,4,1.0,Home/Shelter/Self,CS,OP,None,None,None,0,127,81,1,0,0,1,1,3,3,3,3,0,0,0,0,0,0,0,0,Bipolar Disorders,English,Uninsured/Unknown,Not in Labor Force,High School Diploma/GED,Lives Independently,Missing/Not Reported,1,1
4,5,23,28,Age 26-59,White,Male,0,IPFFS,2,0.0,Satisfactorily Achieved\r\nGoals,OP,FFS,None,None,None,0,14,14,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,Depressive Disorders,English,Medi-Cal + Medicare,Not in Labor Force,High School Diploma/GED,Lives Independently,Missing/Not Reported,0,0


In [28]:
df2.to_csv (r'paths_clean2_AOA_train.csv', index = False, header=True)

In [29]:
#Xtrain = df2
Xtrain = pd.read_csv('paths_clean2_AOA_train.csv')

In [30]:
Xtrain.isnull().values.any()

False

### Data imputation for testing dataset

In [31]:
# count number of rows with missing values
n_miss = Xtest.isnull().sum()
perc = Xtest.isnull().sum()/len(Xtest)*100
df_missing = pd.merge(n_miss.rename('n_miss'), perc.rename('perc_miss'), right_index = True,
               left_index = True)
pd.set_option('display.max_rows', None)
print(df_missing)

                        n_miss  perc_miss
id                           0   0.000000
client_id                    0   0.000000
age                          0   0.000000
age_group                    0   0.000000
race                         0   0.000000
gender                       0   0.000000
dx_group                  1014   4.832253
lang                        98   0.467022
substance                   99   0.471788
insurance                   98   0.467022
emp                         99   0.471788
educ                        99   0.471788
liv                         99   0.471788
justice                      0   0.000000
sexorient                 4364  20.796798
co                          99   0.471788
as_loc                       0   0.000000
as_length_IP                 0   0.000000
IP_history                   0   0.000000
as_close                     0   0.000000
loc1_pre                     0   0.000000
loc2_pre                     0   0.000000
loc3_pre                     0   0

In [32]:
mis_dat_cols = ['dx_group','lang','insurance','emp','educ','liv','sexorient', 'substance', 'co']

In [33]:
Xtest = Xtest.drop(['id', 'client_id'], axis = 1)

In [34]:
k = math.sqrt(Xtest.shape[0])/2
k

72.42927584892728

In [35]:
#Imputing data with KNN
imputer = KNNImputer(n_neighbors=73, weights='uniform', metric='nan_euclidean')
Xtest = pd.DataFrame(imputer.fit_transform(Xtest),columns = Xtest.columns)

In [36]:
Xtest = Xtest.astype(int)
Xtest.isnull().values.any()

False

In [37]:
Xtest_imputedcols = Xtest[mis_dat_cols]
Xtest_othercols = original2.drop([x for x in mis_dat_cols], axis = 1)
Xtest_imputedcols = Xtest_imputedcols.reset_index(drop=True)
Xtest_othercols = Xtest_othercols.reset_index(drop=True)

In [38]:
Xtest_copy = Xtest.copy()
df3 = Xtest_othercols.join(Xtest_imputedcols)
print(df3.shape)
print(df3.isnull().values.any())

(20984, 45)
False


In [39]:
df3['dx_group'] = df3['dx_group'].replace(0,'Anxiety Disorders')
df3['dx_group'] = df3['dx_group'].replace(1,'Bipolar Disorders')
df3['dx_group'] = df3['dx_group'].replace(2,'Depressive Disorders')
df3['dx_group'] = df3['dx_group'].replace(3,'Other/ Excluded')
df3['dx_group'] = df3['dx_group'].replace(4,'Schizophrenia and Other Psychotic Disorders')
df3['dx_group'] = df3['dx_group'].replace(5,'Stressor and Adjustment Disorders')

df3['lang'] = df3['lang'].replace(0,'Asian languages')
df3['lang'] = df3['lang'].replace(1,'English')
df3['lang'] = df3['lang'].replace(2,'Middle Eastern languages')
df3['lang'] = df3['lang'].replace(3,'Other/Unknown')
df3['lang'] = df3['lang'].replace(4,'Spanish')

df3['insurance'] = df3['insurance'].replace(0,'Medi-Cal + Medicare')
df3['insurance'] = df3['insurance'].replace(1,'Private')
df3['insurance'] = df3['insurance'].replace(2,'Uninsured/Unknown')

df3['emp'] = df3['emp'].replace(0,'Competitive Job')
df3['emp'] = df3['emp'].replace(1,'Not Seeking Work')
df3['emp'] = df3['emp'].replace(2,'Not in Labor Force')
df3['emp'] = df3['emp'].replace(3,'Other')
df3['emp'] = df3['emp'].replace(4,'Resident/Inmate of Institution')
df3['emp'] = df3['emp'].replace(5,'Seeking Work')
df3['emp'] = df3['emp'].replace(6,'Unknown')

df3['educ'] = df3['educ'].replace(0,'Associates Degree')
df3['educ'] = df3['educ'].replace(1,'Bachelors Degree')
df3['educ'] = df3['educ'].replace(2,'Doctoral Degree')
df3['educ'] = df3['educ'].replace(3,'High School Diploma/GED')
df3['educ'] = df3['educ'].replace(4,'High School Not Completed')
df3['educ'] = df3['educ'].replace(5,'Masters Degree')
df3['educ'] = df3['educ'].replace(6,'Some College/Vocational Training')
df3['educ'] = df3['educ'].replace(7,'Unknown/Not Reported')

df3['liv'] = df3['liv'].replace(0,'Board & Care')
df3['liv'] = df3['liv'].replace(1,'Homeless')
df3['liv'] = df3['liv'].replace(2,'Institutional')
df3['liv'] = df3['liv'].replace(3,'Justice Related')
df3['liv'] = df3['liv'].replace(4,'Lives Independently')
df3['liv'] = df3['liv'].replace(5,'Other/Unknown')
                                  
df3['sexorient'] = df3['sexorient'].replace(0,'Heterosexual')
df3['sexorient'] = df3['sexorient'].replace(1,'LGBTQ+')
df3['sexorient'] = df3['sexorient'].replace(2,'Missing/Not Reported')

In [40]:
df3.head()

,id,client_id,age,age_group,race,gender,justice,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_AOA_op_7days_pre,svc_AOA_op_30days_pre,svc_AOA_op_60days_pre,svc_AOA_op_90days_pre,svc_AOA_es_7days_pre,svc_AOA_es_30days_pre,svc_AOA_es_60days_pre,svc_AOA_es_90days_pre,svc_AOA_24h_7days_pre,svc_AOA_24h_30days_pre,svc_AOA_24h_60days_pre,svc_AOA_24h_90days_pre,svc_AOA_ip_7days_pre,svc_AOA_ip_30days_pre,svc_AOA_ip_60days_pre,svc_AOA_ip_90days_pre,dx_group,lang,insurance,emp,educ,liv,sexorient,substance,co
0,11,27,45,Age 26-59,Black,Male,1,IPFFS,6,2.0,Home/Shelter/Self,CR,CS,IPFFS,None,None,0,148,9,1,0,0,0,0,2,8,8,8,0,0,0,0,0,0,0,0,Schizophrenia and Other Psychotic Disorders,English,Private,Seeking Work,High School Not Completed,Lives Independently,Heterosexual,1,1
1,13,27,45,Age 26-59,Black,Male,1,IPFFS,1,4.0,Homeless,CR,IPFFS,None,None,None,0,154,5,0,0,0,0,0,0,2,4,4,0,0,0,0,0,0,1,1,Schizophrenia and Other Psychotic Disorders,English,Private,Seeking Work,High School Not Completed,Lives Independently,Heterosexual,1,1
2,15,36,46,Age 26-59,Other,Female,0,IPFFS,3,1.0,Satisfactorily Achieved\r\nGoals,CMSTR,FFS,IPFFS,None,None,0,18,4,1,0,0,3,5,0,0,0,0,0,0,0,0,0,0,0,1,Schizophrenia and Other Psychotic Disorders,English,Medi-Cal + Medicare,Seeking Work,High School Not Completed,Board & Care,Heterosexual,0,0
3,23,36,53,Age 26-59,Other,Female,0,IPCNTY,17,9.0,Home/Shelter/Self,UO,CS,OP,None,None,0,375,13,0,0,5,9,9,5,5,5,10,0,0,0,0,0,0,0,0,Schizophrenia and Other Psychotic Disorders,English,Medi-Cal + Medicare,Not in Labor Force,High School Not Completed,Lives Independently,LGBTQ+,1,1
4,54,63,21,Age <18-25,Hispanic,Male,0,IPCNTY,7,7.0,Home/Shelter/Self,UO,CS,ACT,None,None,0,498,173,1,7,19,33,53,4,4,4,4,0,0,0,0,0,0,0,0,Bipolar Disorders,English,Medi-Cal + Medicare,Not Seeking Work,High School Diploma/GED,Board & Care,Heterosexual,1,1


In [41]:
df3.to_csv (r'paths_clean2_AOA_test.csv', index = False, header=True)

In [42]:
#Xtest = df3
Xtest = pd.read_csv('paths_clean2_AOA_test.csv')

In [43]:
Xtest.isnull().values.any()

False

### Define features for the models

### Numerical features:
1. Age ('age')
2. Length since previous hospitalization ('as_gap_since_former_IP')
3. Length of hospital stay ('as_length_IP')
4. Number of previous hospitalizations ('IP_history')
5. Number of all services before index hospitalization ('svc_pre_all')
6. Number of services before index hospitalization since previous hospitalization ('svc_pre')
7. Number of LOC received before index hospitalization by LOC type and timepoint

In [44]:
numerical_features = ['age', 'as_length_IP', 'IP_history', 'svc_pre_all', 'svc_pre', 'svc_pre_sub']
numerical_features = numerical_features + svc_AOA_timepoint_pre_cols

In [45]:
len(numerical_features)

22

### Categorical features:
1. Demographics (except age): 'age_group','race','gender','dx_group','lang', 'substance','insurance','emp', 'educ','liv','justice', 'sexorient'
2. Type of IP ('as_loc')
3. Type of discharge ('as_close')
4. Last 5 LOCs before index hospitalization
5. Any services before index hospitalization ('new')
6. Any services for 3 years before index hospitalization ('new_3')

In [46]:
# Create dummy variables for non-binary categorical variables
Xtrain = pd.get_dummies(Xtrain, columns=['age_group','race','gender','dx_group','lang', 'insurance','emp', 'educ','liv', 'sexorient',
                       'as_loc', 'as_close','loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre'], drop_first=True)

In [47]:
pd.set_option('display.max_columns', None)
Xtrain.head()

,id,client_id,age,justice,as_length_IP,IP_history,new,svc_pre_all,svc_pre,svc_pre_sub,svc_AOA_op_7days_pre,svc_AOA_op_30days_pre,svc_AOA_op_60days_pre,svc_AOA_op_90days_pre,svc_AOA_es_7days_pre,svc_AOA_es_30days_pre,svc_AOA_es_60days_pre,svc_AOA_es_90days_pre,svc_AOA_24h_7days_pre,svc_AOA_24h_30days_pre,svc_AOA_24h_60days_pre,svc_AOA_24h_90days_pre,svc_AOA_ip_7days_pre,svc_AOA_ip_30days_pre,svc_AOA_ip_60days_pre,svc_AOA_ip_90days_pre,substance,co,age_group_Age 60+,age_group_Age <18-25,race_Black,race_Hispanic,race_Native American,race_Other,race_Unknown,race_White,gender_Male,gender_Other/Unknown,dx_group_Bipolar Disorders,dx_group_Depressive Disorders,dx_group_Other/ Excluded,dx_group_Schizophrenia and Other Psychotic Disorders,dx_group_Stressor and Adjustment Disorders,lang_English,lang_Middle Eastern languages,lang_Other/Unknown,lang_Spanish,insurance_Private,insurance_Uninsured/Unknown,emp_Not Seeking Work,emp_Not in Labor Force,emp_Other,emp_Resident/Inmate of Institution,emp_Seeking Work,emp_Unknown,educ_Bachelors Degree,educ_Doctoral Degree,educ_High School Diploma/GED,educ_High School Not Completed,educ_Masters Degree,educ_Some College/Vocational Training,educ_Unknown/Not Reported,liv_Homeless,liv_Institutional,liv_Justice Related,liv_Lives Independently,liv_Other/Unknown,sexorient_LGBTQ+,sexorient_Missing/Not Reported,as_loc_IPFFS,as_loc_IPLIHP,as_loc_IPST,as_close_Against Medical Advice,as_close_Board & Care,as_close_Client/Family\r\nDissatisfied,as_close_Crisis Residence,as_close_Death/Expired,as_close_Do Not Use / Auto\r\nClosed,as_close_Home/Shelter/Self,as_close_Homeless,as_close_IMD/MHRC,as_close_Incarcerated,as_close_Jail,as_close_Medical Hospital,as_close_Moved,as_close_Nursing Home SNF/ICF,as_close_Other,as_close_Per Court Order,as_close_Psychiatric Hospital,as_close_Required Higher Level\r\nof Care,as_close_Required Lower Level\r\nof Care,as_close_Required Same Level\r\nof Care,as_close_Satisfactorily Achieved\r\nGoals,as_close_State Hospital,as_close_Transfer Medical Hospital,loc1_pre_BHC,loc1_pre_CM,loc1_pre_CMINST,loc1_pre_CMSTR,loc1_pre_CMTRAN,loc1_pre_CR,loc1_pre_CS,loc1_pre_DTC,loc1_pre_DTCTF,loc1_pre_DTR,loc1_pre_EDG,loc1_pre_EPU,loc1_pre_FFS,loc1_pre_IP,loc1_pre_IPCAPS,loc1_pre_IPCNTY,loc1_pre_IPFFS,loc1_pre_IPLIHP,loc1_pre_IPST,loc1_pre_JAIL,loc1_pre_JFS,loc1_pre_LTC,loc1_pre_LTCINST,loc1_pre_LTCRES,loc1_pre_None,loc1_pre_OP,loc1_pre_OPLIHP,loc1_pre_OPR,loc1_pre_PERT,loc1_pre_PREV,loc1_pre_RES,loc1_pre_STRTP,loc1_pre_TBS,loc1_pre_UO,loc1_pre_WRAP,loc2_pre_BHC,loc2_pre_CM,loc2_pre_CMINST,loc2_pre_CMSTR,loc2_pre_CMTRAN,loc2_pre_CR,loc2_pre_CS,loc2_pre_DTC,loc2_pre_DTR,loc2_pre_EDG,loc2_pre_EPU,loc2_pre_ESU,loc2_pre_FFS,loc2_pre_IP,loc2_pre_IPCAPS,loc2_pre_IPCNTY,loc2_pre_IPFFS,loc2_pre_IPLIHP,loc2_pre_IPST,loc2_pre_JAIL,loc2_pre_JFS,loc2_pre_LTC,loc2_pre_LTCINST,loc2_pre_LTCRES,loc2_pre_None,loc2_pre_OP,loc2_pre_OPLIHP,loc2_pre_OPR,loc2_pre_PERT,loc2_pre_PREV,loc2_pre_RES,loc2_pre_STRTP,loc2_pre_TBS,loc2_pre_UO,loc2_pre_WRAP,loc3_pre_BHC,loc3_pre_CM,loc3_pre_CMINST,loc3_pre_CMSTR,loc3_pre_CMTRAN,loc3_pre_CR,loc3_pre_CS,loc3_pre_DTC,loc3_pre_DTCTF,loc3_pre_DTR,loc3_pre_EDG,loc3_pre_EPU,loc3_pre_ESU,loc3_pre_FFS,loc3_pre_IP,loc3_pre_IPCAPS,loc3_pre_IPCNTY,loc3_pre_IPFFS,loc3_pre_IPLIHP,loc3_pre_JAIL,loc3_pre_JFS,loc3_pre_LTC,loc3_pre_LTCINST,loc3_pre_None,loc3_pre_OP,loc3_pre_OPLIHP,loc3_pre_OPR,loc3_pre_PERT,loc3_pre_PREV,loc3_pre_RES,loc3_pre_STRTP,loc3_pre_TBS,loc3_pre_UO,loc3_pre_WRAP,loc4_pre_BHC,loc4_pre_CM,loc4_pre_CMINST,loc4_pre_CMSTR,loc4_pre_CMTRAN,loc4_pre_CR,loc4_pre_CS,loc4_pre_DTC,loc4_pre_DTR,loc4_pre_EDG,loc4_pre_EPU,loc4_pre_ESU,loc4_pre_FFS,loc4_pre_IP,loc4_pre_IPCAPS,loc4_pre_IPCNTY,loc4_pre_IPFFS,loc4_pre_IPLIHP,loc4_pre_JAIL,loc4_pre_JFS,loc4_pre_LTC,loc4_pre_LTCINST,loc4_pre_LTCRES,loc4_pre_None,loc4_pre_OP,loc4_pre_OPLIHP,loc4_pre_OPR,loc4_pre_PERT,loc4_pre_PREV,loc4_pre_RES,loc4_pre_STRTP,loc4_pre_TBS,loc4_pre_UO,loc4_pre_WRAP,loc5_pre_BHC,loc5_pre_CM,loc5_pre_CMINST,loc5_pre_CM

In [48]:
Xtrain.shape

(83936, 264)

In [50]:
dummy_vars = Xtrain.loc[:, 'age_group_Age 60+':'loc5_pre_WRAP']
dummy_vars = list(dummy_vars.columns)
categorical_features = dummy_vars + ['co','substance', 'justice', 'new']

In [51]:
# Select all features
all_features = numerical_features + categorical_features + ['id', 'client_id']
len(all_features)

264

In [52]:
# Create dummy variables for non-binary categorical variables
Xtest = pd.get_dummies(Xtest, columns=['age_group','race','gender','dx_group','lang', 'insurance','emp', 'educ','liv', 'sexorient',
                       'as_loc', 'as_close','loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre'], drop_first=True)

In [53]:
Xtest.shape

(20984, 243)

In [54]:
Xtest_extra_cols = [x for x in all_features if x not in Xtest.columns]
Xtest_extra_cols

['as_close_Client/Family\r\nDissatisfied',
 'as_close_Moved',
 'loc1_pre_DTCTF',
 'loc1_pre_EDG',
 'loc1_pre_IP',
 'loc1_pre_IPST',
 'loc1_pre_LTC',
 'loc2_pre_ESU',
 'loc2_pre_IPST',
 'loc3_pre_BHC',
 'loc3_pre_DTCTF',
 'loc3_pre_ESU',
 'loc4_pre_BHC',
 'loc4_pre_EDG',
 'loc4_pre_ESU',
 'loc4_pre_LTCRES',
 'loc5_pre_BHC',
 'loc5_pre_EDG',
 'loc5_pre_IP',
 'loc5_pre_LTC',
 'loc5_pre_TBS']

In [55]:
Xtest_extra_cols_df = pd.DataFrame(0, index=np.arange(Xtest.shape[0]), columns=Xtest_extra_cols)

In [56]:
Xtest =Xtest.join(Xtest_extra_cols_df)
Xtest.shape

(20984, 264)

In [57]:
Xtrain_extra_cols = [x for x in Xtest.columns if x not in Xtrain.columns]
Xtrain_extra_cols
Xtrain_extra_cols_df = pd.DataFrame(0, index=np.arange(Xtrain.shape[0]), columns=Xtrain_extra_cols)
Xtrain =Xtrain.join(Xtrain_extra_cols_df)
Xtrain.shape

(83936, 264)

In [58]:
name_cols = list(Xtrain.columns)
df_train = pd.merge(Xtrain, ytrain, on='id')
df_test = pd.merge(Xtest, ytest, on='id')

In [59]:
df_train.to_csv (r'paths_AOA_trainData.csv', index = False, header=True)
df_test.to_csv (r'paths_AOA_testData.csv', index = False, header=True)

### Archive codes

In [ ]:
'''
# Map values
dx_group_dict = {0:'Anxiety Disorders', 1:'Bipolar Disorders', 2:'Depressive Disorders', 3: 'Other/ Excluded',
                4: 'Schizophrenia and Other Psychotic Disorders', 5: 'Stressor and Adjustment Disorders'}
lang_dict = {0:'Asian languages', 1:'English', 2:'Middle Eastern languages', 3:'Other/Unknown', 4:'Spanish'}
insurance_dict = {0:'Medi-Cal + Medicare', 1:'Private', 2:'Uninsured/Unknown'}
emp_dict = {0:'Competitive Job', 1:'Not Seeking Work', 2:'Not in Labor Force', 3:'Other', 4:'Resident/Inmate of Institution',
           5:'Seeking Work', 6:'Unknown'}
educ_dict = {0:'Associates Degree', 1:'Bachelors Degree', 2:'High School Diploma/GED', 3:'High School Not Completed',
            4:'Masters Degree', 5:'Some College/Vocational Training', 6:'Unknown/Not Reported'}
liv_dict = {0:'Board & Care', 1:'Homeless', 2:'Institutional', 3:'Justice Related', 4:'Lives Independently', 5:'Other/Unknown'}
sexorient_dict = {0:'Bisexual', 1:'Decline to State', 2:'Deferred', 3:'Gay Male', 4:'Heterosexual', 5:'Lesbian',
                 6:'Missing', 7:'Other', 8:'Questioning', 9:'Transgender'}
df2['dx_group'] = df2['dx_group'].map(dx_group_dict)
df2['lang'] = df2['lang'].map(lang_dict)
df2['insurance'] = df2['insurance'].map(insurance_dict)
df2['emp'] = df2['emp'].map(emp_dict)
df2['educ'] = df2['educ'].map(educ_dict)
df2['liv'] = df2['liv'].map(liv_dict)
df2['sexorient'] = df2['sexorient'].map(sexorient_dict)
'''

In [ ]:
'''
#Impute missing value with mean
for col in numerical_features:
    df[col].fillna(df[col].mean(), inplace=True)
'''

In [ ]:
'''
# Create dummy variables for non-binary categorical variables
df = pd.get_dummies(df, columns=['age_group','race','gender','dx_group','lang', 'insurance','emp', 'educ','liv', 'sexorient',
                       'as_loc', 'as_close','loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre'], drop_first=True)

'''

In [ ]:
'''
# impute missing values of categorical features with the most frequent object
for col in categorical_features:
    df[col].fillna(df[col].value_counts().index[0], inplace=True)
'''

In [ ]:
'''
svc_AOA_timepoint_pre_cols = [ 'svc_AOA_op_7days_pre', 'svc_AOA_op_30days_pre', 'svc_AOA_op_60days_pre', 'svc_AOA_op_90days_pre',
                     'svc_AOA_es_7days_pre', 'svc_AOA_es_30days_pre', 'svc_AOA_es_60days_pre', 'svc_AOA_es_90days_pre',
                     'svc_AOA_fs_7days_pre', 'svc_AOA_fs_30days_pre', 'svc_AOA_fs_60days_pre', 'svc_AOA_fs_90days_pre',
                     'svc_AOA_24h_7days_pre', 'svc_AOA_24h_30days_pre', 'svc_AOA_24h_60days_pre', 'svc_AOA_24h_90days_pre',
                     'svc_AOA_ip_7days_pre', 'svc_AOA_ip_30days_pre', 'svc_AOA_ip_60days_pre', 'svc_AOA_ip_90days_pre']

'''

In [ ]:
'''
# Calculate the prevalence of rehospitalization
admissions = len(df)
counts = df.groupby('output')['id'].count().values
prevalence = counts[1]/(counts[0]+counts[1])
print("The total hospitalization is ", admissions)
print("The prevalence of 90-day rehospitalization is ", prevalence)
print("The absolute number of rehospitalized patients is ", counts[1])

'''

In [ ]:
'''
# Create an array with colors
colors = ["#6495ed", "#ff6347"]
# Set custom color palette
sns.set_palette(sns.color_palette(colors))

# create a column for more informative leend plotting
df.loc[df['output'] == 1, 'Rehospitalization'] = 'Yes'
df.loc[df['output'] == 0, 'Rehospitalization'] = 'No'

'''

In [ ]:


#df2.reset_index(inplace=True)
#df2 = df2.rename(columns = {'index':'id'})
#df2['id'] =df2['id'] + 1

In [ ]:
'''
# Recode sexual orientation
df2['sexorient'] = df2['sexorient'].replace(['Bisexual','Gay Male','Transgender','Lesbian', 'Other','Questioning', 'Transgender'],'LGBTQ+')
df2['sexorient'] = df2['sexorient'].replace(['Decline to State','Deferred','Missing'],'Missing/Not Reported')
print(df2[['client_id','sexorient']].groupby('client_id').max().sexorient.value_counts())

'''

In [ ]:
'''
#df2 = original.copy()
df2 = original.where(~mask, Xtrain)
'''

In [ ]:
'''
loc_AOA_timepoint_pre_cols = ['loc_pre_7days_ACT', 
'loc_pre_30days_ACT',
'loc_pre_60days_ACT',
'loc_pre_90days_ACT',
'loc_pre_7days_CM',
'loc_pre_30days_CM',
'loc_pre_60days_CM',
'loc_pre_90days_CM',
'loc_pre_7days_CMINST',
'loc_pre_30days_CMINST',
'loc_pre_60days_CMINST',
'loc_pre_90days_CMINST',
'loc_pre_7days_CMSTR',
'loc_pre_30days_CMSTR',
'loc_pre_60days_CMSTR',
'loc_pre_90days_CMSTR',
'loc_pre_7days_CMTRAN',
'loc_pre_30days_CMTRAN',
'loc_pre_60days_CMTRAN',
'loc_pre_90days_CMTRAN',
'loc_pre_7days_FFS',
'loc_pre_30days_FFS',
'loc_pre_60days_FFS',
'loc_pre_90days_FFS',
'loc_pre_7days_OP',
'loc_pre_30days_OP',
'loc_pre_60days_OP',
'loc_pre_90days_OP',
'loc_pre_7days_PREV',
'loc_pre_30days_PREV',
'loc_pre_60days_PREV',
'loc_pre_90days_PREV',
'loc_pre_7days_CS',
'loc_pre_30days_CS',
'loc_pre_60days_CS',
'loc_pre_90days_CS',
'loc_pre_7days_PERT',
'loc_pre_30days_PERT',
'loc_pre_60days_PERT',
'loc_pre_90days_PERT',
'loc_pre_7days_UO',
'loc_pre_30days_UO',
'loc_pre_60days_UO',
'loc_pre_90days_UO',                              
'loc_pre_7days_IPCNTY',
'loc_pre_30days_IPCNTY',
'loc_pre_60days_IPCNTY',
'loc_pre_90days_IPCNTY',
'loc_pre_7days_IPFFS',
'loc_pre_30days_IPFFS',
'loc_pre_60days_IPFFS',
'loc_pre_90days_IPFFS',
'loc_pre_7days_JAIL',
'loc_pre_30days_JAIL',
'loc_pre_60days_JAIL',
'loc_pre_90days_JAIL',
'day_loc_pre_7days_CR',
'day_loc_pre_30days_CR',
'day_loc_pre_60days_CR',
'day_loc_pre_90days_CR',
'day_loc_pre_7days_EDG',
'day_loc_pre_30days_EDG',
'day_loc_pre_60days_EDG',
'day_loc_pre_90days_EDG',
'day_loc_pre_7days_LTC',
'day_loc_pre_30days_LTC',
'day_loc_pre_60days_LTC',
'day_loc_pre_90days_LTC',
'day_loc_pre_7days_LTCINST',
'day_loc_pre_30days_LTCINST',
'day_loc_pre_60days_LTCINST',
'day_loc_pre_90days_LTCINST',
'day_loc_pre_7days_LTCRES',
'day_loc_pre_30days_LTCRES',
'day_loc_pre_60days_LTCRES',
'day_loc_pre_90days_LTCRES',
'day_loc_pre_7days_RES',
'day_loc_pre_30days_RES',
'day_loc_pre_60days_RES',
'day_loc_pre_90days_RES']
'''

In [ ]:
'''
#df.rename(columns = {'svc_AOA_ip_90days_postdc_bi':'output'}, inplace = True)
#output_col = ['output']
#drop_cols = ['']
'''